In [9]:
import gradio as gr
import requests
import re

# Azure OpenAI 설정
AZURE_OPENAI_ENDPOINT = 'https://1507-ai.openai.azure.com/'
AZURE_OPENAI_API_KEY = '22646ec32d23457ca3ed253eefc5342f'
DEPLOYMENT_NAME = '1507-gpt4o'

# Azure OpenAI API 호출 함수
def chatgpt_response(prompt, history, lang):  
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # 언어 선택에 따른 시스템 메시지 변경
    system_message = {
        "ko-kr": "너는 서울 구로구의 쉼터에 대한 전문가야. 유저가 작성한 장소에서 어느 곳이 가까운지, 또 쉼터에 대한 정보를 알려줘. 모든 질문에 한국어로 답변해.",  # 한국어 시스템 메시지
        "en-un": "I am a shelter expert in Guro-gu, Seoul. Notifies users of the nearest shelter to a specific location. Respond to the user's query in English regardless of the input language.",  # 영어 시스템 메시지 (질문이 어떤 언어로 입력되든 영어로 답변하도록 지시)
        "ja-jp": "私はソウル区の避難所の専門家です。特定の場所に最も近い避難所をユーザーに通知します。入力言語に関係なく、すべての質問に対して日本語で回答してください。",  # 일본어 시스템 메시지
    }

    messages = []

    # System 메시지 추가
    messages.append({
        "role": "system",
        "content": system_message.get(lang, system_message["ko-kr"])  # 선택된 언어에 따라 시스템 메시지를 설정, 기본값은 한국어
    })

    # Assistant의 이전 대화 추가
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": text  # 이전 GPT 응답 추가
            })

    # User 입력 추가
    messages.append({
        "role": "user",
        "content": prompt  # 사용자의 입력 추가
    })

    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }

    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  
    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))  # 대화 기록에 사용자 입력과 GPT 응답 추가
    return bot_response, history

# 텍스트에서 한국어, 영어, 숫자만 남기는 함수
def filter_text(text):
    filtered_text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)  # 정규식을 이용해 한국어, 영어, 숫자 이외의 문자를 제거
    # filtered_text = re.sub(r'[!@#$%^&*()_+-=[]{}:;,./<>?/*`~]', ' ', text)
    return filtered_text

# Azure TTS 설정 및 호출 함수
def get_token():
    endpoint = 'https://eastus.api.cognitive.microsoft.com/sts/v1.0/issueToken'
    api_key = '5fb3d580ca744ae7a95b9fbe10307ec3'

    headers = {
        "Ocp-Apim-Subscription-Key": api_key
    }
    
    response = requests.post(endpoint, headers=headers)
    
    if response.status_code == 200:
        access_token = response.text
        return access_token
    else:
        return ''

def request_tts(text, lang):
    endpoint = 'https://eastus.tts.speech.microsoft.com/cognitiveservices/v1'
    access_token = get_token()

    headers = {
        "X-Microsoft-OutputFormat": "riff-24khz-16bit-mono-pcm",
        "Content-Type": "application/ssml+xml",
        "User-Agent": "testForEducation",
        "Authorization": f"Bearer {access_token}"
    }

    # 언어별 목소리 설정
    voice_settings = {
        "ko-kr": "ko-KR-SunHiNeural",  # 한국어 목소리
        "en-un": "en-US-AriaNeural",  # 영어 목소리
        "ja-jp": "ja-JP-NanamiNeural" # 일본어 목소리
    }
    
    # 필터된 텍스트 사용 및 TTS 요청 데이터 생성
    voice = voice_settings.get(lang, voice_settings["ko-kr"])  # 선택된 언어에 맞는 목소리를 설정, 기본값은 한국어
    data = f"""
    <speak version='1.0' xml:lang='{lang}'><voice xml:lang='{lang}' xml:gender='Female' name='{voice}'>
        {text}
    </voice></speak>
    """

    response = requests.post(endpoint, headers=headers, data=data)
    
    if response.status_code == 200:
        file_name = 'response_audio.wav'
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        return file_name
    else:
        return None

def chatgpt_with_tts(prompt, history, lang):
    response_text, updated_history = chatgpt_response(prompt, history, lang)
    
    # 응답 텍스트 필터링
    filtered_response_text = filter_text(response_text)
    
    # TTS 변환
    audio_file = request_tts(filtered_response_text, lang)
    return audio_file, updated_history

# Gradio 인터페이스 설정
with gr.Blocks(css=".title {text-align: center; color: #f5f0f0; font-family: 'Arial';} .footer {text-align: right; color: #7f8c8d; font-family: 'Arial';}") as demo:
    gr.Markdown("<h1 align='center'>ChatGPT with TTS Demo</h1>")
    lang_dropdown = gr.Dropdown(["ko-kr", "en-un", "ja-jp"], label="언어 선택")  # 언어 선택 Dropdown 추가
    chatbot = gr.Chatbot()
    with gr.Column():
        with gr.Row():
            user_input = gr.Textbox(show_label=False, placeholder="Type your message here...", scale=3)
            submit_btn = gr.Button("Send", scale=1, variant="primary")
        with gr.Row():
            clear_btn = gr.Button("Clear")
        with gr.Row():
            output_audio = gr.Audio(label="Output Audio", type="filepath", interactive=False)

    gr.Markdown("<h3 class='footer'>제작자: CheongYa</h3>")
    gr.Markdown("<h3 class='footer'>최종 수정일: 2024-08-02</h3>")

    # 버튼 클릭 시 이벤트 처리
    submit_btn.click(chatgpt_with_tts, [user_input, chatbot, lang_dropdown], [output_audio, chatbot])
    clear_btn.click(lambda: [None, None, None], None, [chatbot, output_audio, user_input], queue=False)
    # 엔터 처리
    user_input.submit(chatgpt_with_tts, [user_input, chatbot, lang_dropdown], [output_audio, chatbot])

# 인터페이스 실행
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7873

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
